# LSTM multivariate time forecasting using TensorFlow / Keras

In [1]:
import pandas as pd
import numpy as np
np.random.seed(524)

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, LSTM

from typing import Tuple, Sequence

Using TensorFlow backend.


### Data Aggregation / Feature Engineering
this time using methods in https://www.kaggle.com/the1owl/surprise-me kernel instead of from the LightGBM kernel; this kernel looks at reservations in a way that seems a lot more useful

In [2]:
data = {
    'tra': pd.read_csv('data/air/air_visit_data.csv'),
    'as': pd.read_csv('data/air/air_store_info.csv'),
    'hs': pd.read_csv('data/hpg/hpg_store_info.csv'),
    'ar': pd.read_csv('data/air/air_reserve.csv'),
    'hr': pd.read_csv('data/hpg/hpg_reserve.csv'),
    'id': pd.read_csv('data/store_id_relation.csv'),
    'tes': pd.read_csv('sample_submission.csv'),
    'hol': pd.read_csv('data/date_info.csv').rename(columns={'calendar_date':'visit_date'})
}

# Merge store id relation data onto HPG data
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

#### splitting up datetimes

In [3]:
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date    
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    data[df] = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date'})

In [4]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

#### engineering visitor statistics on a store by store basis

In [5]:
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

#### label encoding restaurant data and holiday data

In [6]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
lbl = LabelEncoder()
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

#### setting up train and test data

In [7]:
train = pd.merge(data['tra'], stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(data['tes'], stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

# deal with missing data
train = train.fillna(-1)
test = test.fillna(-1)

#### defining error function

In [8]:
def rmsle(y, pred):
    return mean_squared_error(y, pred) ** 0.5

In [9]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0


## 1: Visitors as a feature to fit LSTM

normalize

In [10]:
train = train.sort_values('visit_date')
values = np.log1p(train['visitors'].values).reshape(-1,1)
values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)


split into train and test sets

In [11]:
train_size = int(len(scaled) * 0.8)
test_size = len(scaled) - train_size

V_train, V_test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
print(len(V_train), len(V_test))

201686 50422


convert an array of values into dataset matrix

In [12]:
def create_dataset(dataset: Sequence, look_back=1) -> Tuple[np.array, np.array]:
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [13]:
look_back = 1
trainX, trainY = create_dataset(V_train, look_back)
testX, testY = create_dataset(V_test, look_back)

201685
50421


reshape X for training

In [14]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

train with 3 epochs

In [15]:
model = Sequential()
model.add(LSTM(4, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
history = model.fit(trainX, trainY, epochs=3, batch_size=100,
                            validation_data=(testX, testY), verbose=1, shuffle=False)

Train on 201685 samples, validate on 50421 samples
Epoch 1/3
201685/201685 [==============================] - 11s 56us/step - loss: 0.0221 - val_loss: 0.0186
Epoch 2/3
201685/201685 [==============================] - 9s 47us/step - loss: 0.0176 - val_loss: 0.0186
Epoch 3/3
201685/201685 [==============================] - 9s 47us/step - loss: 0.0176 - val_loss: 0.0186


make prediction and apply invert scaling

In [16]:
yhat = model.predict(testX)

yhat_inverse = scaler.inverse_transform(yhat.reshape(-1, 1))
testY_inverse = scaler.inverse_transform(testY.reshape(-1, 1))

calculate RMSLE

In [17]:
test_rmsle = rmsle(testY_inverse, yhat_inverse)
print('Test RMSLE: %.3f' % test_rmsle)

Test RMSLE: 0.831


## 2: Multivariate forecasting

functions from https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras

Training model using all features

In [18]:
train = train.sort_values('visit_date')
target_train = np.log1p(train['visitors'].values)

col = [c for c in train if c not in ['id', 'air_store_id', 'visitors']]

train = train[col]
train.set_index('visit_date', inplace=True)

train.head()

,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
visit_date,,,,,,,,,,,,,,,,
2016-01-01,4,2016,1,2.0,10.382353,10.0,20.0,68.0,2.0,3.0,33.581941,130.348436,-1.0,-1.0,-1.0,-1.0
2016-01-01,4,2016,1,3.0,18.656716,18.0,48.0,67.0,2.0,77.0,35.693840,139.703549,-1.0,-1.0,-1.0,-1.0
2016-01-01,4,2016,1,10.0,41.855072,41.0,83.0,69.0,7.0,66.0,35.735623,139.651658,-1.0,-1.0,-1.0,-1.0
2016-01-01,4,2016,1,1.0,4.322034,3.0,13.0,59.0,1.0,92.0,34.766093,135.628100,-1.0,-1.0,-1.0,-1.0
2016-01-01,4,2016,1,15.0,32.985507,31.0,71.0,69.0,6.0,30.0,34.737597,135.341564,-1.0,-1.0,-1.0,-1.0


In [19]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

normalize features

In [20]:
train['visitors'] = target_train
values = train.values
values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

reframe as supervised

In [21]:
reframed = series_to_supervised(scaled, 1, 1)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var8(t),var9(t),var10(t),var11(t),var12(t),var13(t),var14(t),var15(t),var16(t),var17(t)
1,0.666667,0.0,0.0,0.022556,0.076686,0.071895,0.023918,0.985714,0.214286,0.038835,...,0.971429,0.214286,0.757282,0.815045,0.968543,0.0,0.0,0.0,0.0,0.465644
2,0.666667,0.0,0.0,0.030075,0.132432,0.124183,0.055809,0.971429,0.214286,0.757282,...,1.000000,0.571429,0.650485,0.815973,0.968186,0.0,0.0,0.0,0.0,0.280179
3,0.666667,0.0,0.0,0.082707,0.288725,0.274510,0.095672,1.000000,0.571429,0.650485,...,0.857143,0.142857,0.902913,0.794438,0.940490,0.0,0.0,0.0,0.0,0.205894
4,0.666667,0.0,0.0,0.015038,0.035856,0.026144,0.015945,0.857143,0.142857,0.902913,...,1.000000,0.500000,0.300971,0.793805,0.938517,0.0,0.0,0.0,0.0,0.351724
5,0.666667,0.0,0.0,0.120301,0.228969,0.209150,0.082005,1.000000,0.500000,0.300971,...,1.000000,0.142857,0.844660,0.815762,0.968633,0.0,0.0,0.0,0.0,0.150594


drop unnecessary columns (those without significant correlation)

In [22]:
reframed.iloc[[i for i in range(17,33)]].head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var8(t),var9(t),var10(t),var11(t),var12(t),var13(t),var14(t),var15(t),var16(t),var17(t)
18,0.666667,0.0,0.0,0.015038,0.114533,0.088235,0.055809,0.785714,0.642857,0.233010,...,0.957143,0.142857,0.699029,0.814280,0.968494,0.0,0.0,0.0,0.0,0.150594
19,0.666667,0.0,0.0,0.015038,0.066658,0.065359,0.025057,0.957143,0.142857,0.699029,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.351724
20,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.571429,0.009709,0.768297,0.904452,0.0,0.0,0.0,0.0,0.465644
21,0.666667,0.0,0.0,0.105263,0.285698,0.287582,0.099089,1.000000,0.571429,0.009709,...,0.971429,0.857143,0.728155,0.814333,0.968547,0.0,0.0,0.0,0.0,0.247198
22,0.666667,0.0,0.0,0.015038,0.112522,0.084967,0.060364,0.971429,0.857143,0.728155,...,1.000000,0.357143,0.271845,0.792861,0.937528,0.0,0.0,0.0,0.0,0.351724


In [23]:
reframed.drop(reframed.columns[[i for i in range(17,33)]], axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var17(t)
1,0.666667,0.0,0.0,0.022556,0.076686,0.071895,0.023918,0.985714,0.214286,0.038835,0.768135,0.904147,0.0,0.0,0.0,0.0,0.378435,0.465644
2,0.666667,0.0,0.0,0.030075,0.132432,0.124183,0.055809,0.971429,0.214286,0.757282,0.815045,0.968543,0.0,0.0,0.0,0.0,0.465644,0.280179
3,0.666667,0.0,0.0,0.082707,0.288725,0.274510,0.095672,1.000000,0.571429,0.650485,0.815973,0.968186,0.0,0.0,0.0,0.0,0.280179,0.205894
4,0.666667,0.0,0.0,0.015038,0.035856,0.026144,0.015945,0.857143,0.142857,0.902913,0.794438,0.940490,0.0,0.0,0.0,0.0,0.205894,0.351724
5,0.666667,0.0,0.0,0.120301,0.228969,0.209150,0.082005,1.000000,0.500000,0.300971,0.793805,0.938517,0.0,0.0,0.0,0.0,0.351724,0.150594


split test and train sets

In [24]:
values = reframed.values
n_train_days = int(len(values) * 0.7)
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# Split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# Reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(176474, 1, 17) (176474,) (75633, 1, 17) (75633,)


train LSTM

In [25]:
multi_model = Sequential()
multi_model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
multi_model.add(Dense(1))
multi_model.compile(loss='mse', optimizer='adam')
multi_history = multi_model.fit(train_X, train_y, epochs=15,
                                batch_size=100, validation_data=(test_X, test_y),
                                verbose=1, shuffle=False)

Train on 176474 samples, validate on 75633 samples
Epoch 1/15
176474/176474 [==============================] - 18s 104us/step - loss: 0.0182 - val_loss: 0.0202
Epoch 2/15
176474/176474 [==============================] - 17s 96us/step - loss: 0.0174 - val_loss: 0.0192
Epoch 3/15
176474/176474 [==============================] - 17s 96us/step - loss: 0.0172 - val_loss: 0.0192
Epoch 4/15
176474/176474 [==============================] - 16s 88us/step - loss: 0.0171 - val_loss: 0.0189
Epoch 5/15
176474/176474 [==============================] - 15s 83us/step - loss: 0.0171 - val_loss: 0.0186
Epoch 6/15
176474/176474 [==============================] - 15s 88us/step - loss: 0.0171 - val_loss: 0.0183
Epoch 7/15
176474/176474 [==============================] - 17s 95us/step - loss: 0.0171 - val_loss: 0.0180
Epoch 8/15
176474/176474 [==============================] - 16s 90us/step - loss: 0.0170 - val_loss: 0.0179
Epoch 9/15
176474/176474 [==============================] - 18s 101us/step - loss: 0

make prediction and apply invert scaling

In [26]:
yhat = multi_model.predict(test_X)

test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# Invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# Invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

RSMLE

In [27]:
multi_rmsle = rmsle(inv_y, inv_yhat)
print('Test RMSLE: %.3f' % multi_rmsle) #0.8

Test RMSLE: 0.800
